In [1]:
import pandas as pd
# Set max columns to display
pd.set_option('display.max_columns', None)

import numpy as np
from sklearn import preprocessing as pre
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score



C:\Users\austinsh\AppData\Local\Temp\ipykernel_16456\1304120184.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:

# # Importing CSV files
# df_CDunit = pd.read_csv('https://raw.githubusercontent.com/saust1/Project-OptiC4/main/1%20Preprocess/Continuous%20Data/cont_554Data_clean.csv')
# df_AlCon = pd.read_csv('https://raw.githubusercontent.com/saust1/Project-OptiC4/main/1%20Preprocess/Continuous%20Data/cont_425Data_clean.csv')
# df_FB554 = pd.read_csv('https://raw.githubusercontent.com/saust1/Project-OptiC4/main/1%20Preprocess/Continuous%20Data/cont_unitData_clean.csv')


In [3]:
# # Importing CSV files
# bordeCode directory
df_CDunit = pd.read_csv(r'C:\Users\austinsh\Project-OptiC4\IV Optimize\Testing Data\Continuous Data 2023\cont_unitData_clean.csv')
df_AlCon = pd.read_csv(r'C:\Users\austinsh\Project-OptiC4\IV Optimize\Testing Data\Continuous Data 2023\cont_425Data_clean.csv')
df_FB554 = pd.read_csv(r'C:\Users\austinsh\Project-OptiC4\IV Optimize\Testing Data\Continuous Data 2023\cont_554Data_clean.csv')



In [4]:
print(df_CDunit.describe())
print(df_AlCon.describe())
print(df_FB554.describe())


           DI55102      DI55152      DI55580      FC55003      FC55009  \
count  4863.000000  4863.000000  4863.000000  4863.000000  4863.000000   
mean      0.825533     1.022945     0.970547  5915.277415  1110.353856   
std       0.055241     0.025410     0.036531   704.302443   366.916357   
min       0.750013     0.965978     0.873708  3495.310000    59.931500   
25%       0.794854     1.001015     0.941198  5447.270000   865.216500   
50%       0.794860     1.026990     0.983524  5890.350000  1084.950000   
75%       0.862055     1.046290     0.998387  6403.680000  1326.785000   
max       0.999487     1.071690     1.030410  8369.130000  2244.140000   

            FC55552      FC55569      FC55576     FFC55553     FFC55555  \
count   4863.000000  4863.000000  4863.000000  4863.000000  4863.000000   
mean   34708.236973  6521.006597   374.621914     0.918578     0.786579   
std     3406.599218   336.992674   189.402749     0.045267     0.013370   
min    23523.500000  5477.780000 

In [5]:
print("Data type for 'Date' column in df_CDunit:", df_CDunit['Date'].dtypes)
print("Data type for 'Date' column in df_FB554:", df_FB554['Date'].dtypes)
print("Data type for 'Date' column in df_AlCon:", df_AlCon['Date'].dtypes)


Data type for 'Date' column in df_CDunit: object
Data type for 'Date' column in df_FB554: object
Data type for 'Date' column in df_AlCon: object


In [6]:
df_CDunit['Date'] = pd.to_datetime(df_CDunit['Date'], errors='coerce')
df_FB554['Date'] = pd.to_datetime(df_FB554['Date'], errors='coerce')
df_AlCon['Date'] = pd.to_datetime(df_AlCon['Date'], errors='coerce')


In [7]:
print("Data type for 'Date' column in df_CDunit:", df_CDunit['Date'].dtypes)
print("Data type for 'Date' column in df_FB554:", df_FB554['Date'].dtypes)
print("Data type for 'Date' column in df_AlCon:", df_AlCon['Date'].dtypes)


Data type for 'Date' column in df_CDunit: datetime64[ns]
Data type for 'Date' column in df_FB554: datetime64[ns]
Data type for 'Date' column in df_AlCon: datetime64[ns]


In [8]:
print(df_CDunit.columns)
print(df_FB554.columns)
print(df_AlCon.columns)


Index(['Date', 'DI55102', 'DI55152', 'DI55580', 'FC55003', 'FC55009',
       'FC55552', 'FC55569', 'FC55576', 'FFC55553', 'FFC55555', 'LC55557',
       'LC90366', 'LC90368', 'PI55020', 'TC55552', 'TI55021'],
      dtype='object')
Index(['Date', 'Butanol', 'Decanol', 'Ethanol', 'Hexanol', 'Octanol'], dtype='object')
Index(['Date', '425_pct_Al', 'Al2O3', 'M_Value', 'C4_pct_Eth', 'C4_pct_H2O',
       'HydWtr_pct_Ammonia', 'C4_pct_Hex', 'HydWtr_Na2O'],
      dtype='object')


In [9]:
def apply_rolling_average_to_df(df, rolling_size):
    # Ensure 'Date' is the index if it's not already
    if df.index.name != 'Date':
        df = df.set_index('Date')

    # Apply rolling average to all columns
    rolled_df = df.rolling(window=rolling_size, min_periods=1).mean()

    # Reset index to make 'Date' a column again
    rolled_df = rolled_df.reset_index()

    return rolled_df



In [10]:
def apply_time_shift_by_hours(df, shift_hours):
    """
    Shifts the DataFrame's datetime index by the specified number of hours.

    :param df: DataFrame with 'Date' as its datetime index or column.
    :param shift_hours: Number of hours to shift. Can be positive (forward) or negative (backward).
    :return: Shifted DataFrame.
    """
    # Convert 'Date' to datetime and set as index if it's not already
    if df.index.name != 'Date':
        df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
        df = df.set_index('Date')

    # Ensure the index is a DatetimeIndex
    df.index = pd.to_datetime(df.index)

    # Shift the DataFrame's index by the specified number of hours
    df.index = df.index + pd.Timedelta(hours=shift_hours)

    return df

In [11]:
# # Usage Examples
# shift_hours_AlCon = 1  # Negative shift for df_AlCon (e.g., -5 hours backward)
# shift_hours_FB554 = 5   # Positive shift for df_FB554 (e.g., 5 hours forward)

# shifted_df_AlCon = apply_time_shift_by_hours(df_AlCon, shift_hours_AlCon)
# print("Shifted df_AlCon:")
# print(shifted_df_AlCon.head())

# shifted_df_FB554 = apply_time_shift_by_hours(df_FB554, shift_hours_FB554)
# print("\nShifted df_FB554:")
# print(shifted_df_FB554.head())

In [12]:
def join_df_FB554_to_df_CDunit(df_CDunit, df_FB554):
    # Reset index if 'Date' is the index
    if df_CDunit.index.name == 'Date':
        df_CDunit = df_CDunit.reset_index()
    if df_FB554.index.name == 'Date':
        df_FB554 = df_FB554.reset_index()

    # Ensure 'Date' columns are datetime objects and sort DataFrames
    df_CDunit['Date'] = pd.to_datetime(df_CDunit['Date'], errors='coerce')
    df_FB554['Date'] = pd.to_datetime(df_FB554['Date'], errors='coerce')

    df_CDunit = df_CDunit.dropna(subset=['Date']).sort_values('Date')
    df_FB554 = df_FB554.dropna(subset=['Date']).sort_values('Date')

    # Perform merge_asof
    combined_df = pd.merge_asof(df_FB554, df_CDunit, on='Date', direction='nearest')

    return combined_df

def join_df_AlCon_to_combined_df(combined_df, df_AlCon):
    # Reset index if 'Date' is the index
    if combined_df.index.name == 'Date':
        combined_df = combined_df.reset_index()
    if df_AlCon.index.name == 'Date':
        df_AlCon = df_AlCon.reset_index()

    # Ensure 'Date' columns are datetime objects and sort DataFrames
    combined_df['Date'] = pd.to_datetime(combined_df['Date'], errors='coerce')
    df_AlCon['Date'] = pd.to_datetime(df_AlCon['Date'], errors='coerce')

    combined_df = combined_df.dropna(subset=['Date']).sort_values('Date')
    df_AlCon = df_AlCon.dropna(subset=['Date']).sort_values('Date')

    # Perform merge_asof
    combined_df_all = pd.merge_asof(df_AlCon, combined_df, on='Date', direction='nearest')
    
    return combined_df_all


In [13]:
print("Data type for 'Date' column in df_CDunit:", df_CDunit['Date'].dtypes)
print("Data type for 'Date' column in df_FB554:", df_FB554['Date'].dtypes)
print("Data type for 'Date' column in df_AlCon:", df_AlCon['Date'].dtypes)



Data type for 'Date' column in df_CDunit: datetime64[ns]
Data type for 'Date' column in df_FB554: datetime64[ns]
Data type for 'Date' column in df_AlCon: datetime64[ns]


In [14]:
def process_data_limited():
    # Apply the specific rolling average directly
    rolled_df_CDunit = apply_rolling_average_to_df(df_CDunit, 8)
    rolled_df_FB554 = apply_rolling_average_to_df(df_FB554, 4)
    rolled_df_AlCon = apply_rolling_average_to_df(df_AlCon, 2)

    # Apply the specific time shifts directly
    rolled_df_AlCon_shifted = apply_time_shift_by_hours(rolled_df_AlCon, -1) # Assuming apply_time_shift_by_hours handles negative shifts correctly
    rolled_df_FB554_shifted = apply_time_shift_by_hours(rolled_df_FB554, 1)

    # Combine df_CDunit and df_FB554 to create combined_df
    combined_df = join_df_FB554_to_df_CDunit(rolled_df_CDunit, rolled_df_FB554_shifted)

    # Combine combined_df with rolled_df_AlCon to create combined_df_all
    combined_df_all = join_df_AlCon_to_combined_df(combined_df, rolled_df_AlCon_shifted)

    # At this point, combined_df_all is the DataFrame with the data processed by the specified shifts and averages
    return combined_df_all


In [15]:
# Make sure all your helper functions and initial DataFrames (df_CDunit, df_FB554, df_AlCon) are correctly defined

# Now, call the modified process_data function to get the processed DataFrame
final_dataset = process_data_limited()

# Inspect the final_dataset
print(final_dataset.head())  # Print the first few rows to inspect the dataset


                 Date  425_pct_Al     Al2O3   M_Value  C4_pct_Eth  C4_pct_H2O  \
0 2023-01-01 03:00:00    6.159550  10.76240  3.901390    2.077530    21.50120   
1 2023-01-01 09:00:00    6.159750  10.76285  3.893505    2.123750    21.55375   
2 2023-01-01 10:00:00    6.159985  10.76340  3.884305    2.177670    21.61510   
3 2023-01-01 18:00:00    6.160285  10.76415  3.843085    2.246995    21.69400   
4 2023-01-01 23:00:00    6.160715  10.76520  3.772120    2.347140    21.80795   

   HydWtr_pct_Ammonia  C4_pct_Hex  HydWtr_Na2O   Butanol   Decanol    Ethanol  \
0            1.229850    0.387521     3.528950  3.268330  1.755000  12.803300   
1            1.212805    0.399834     3.239975  3.569165  1.614165  13.817500   
2            1.190910    0.414198     2.880790  3.569165  1.614165  13.817500   
3            1.147270    0.432667     2.248915  3.674583  1.750417  14.145400   
4            1.084235    0.459344     1.336206  3.675000  1.855417  13.964575   

    Hexanol   Octanol   DI

In [16]:
# Inspect the final_dataset
print(final_dataset.describe())  # Print the first few rows to inspect the dataset

                                Date   425_pct_Al        Al2O3      M_Value  \
count                           4863  4863.000000  4863.000000  4863.000000   
mean   2023-07-18 16:10:46.267736064     6.222890    11.538018     3.633789   
min              2023-01-01 03:00:00     5.020410    10.762400     3.340825   
25%              2023-04-22 19:00:00     6.124507    11.198225     3.567775   
50%              2023-07-24 21:00:00     6.266860    11.561850     3.626065   
75%              2023-10-29 15:30:00     6.378610    11.939825     3.706710   
max              2023-12-31 22:00:00     6.850010    12.201400     3.901390   
std                              NaN     0.266864     0.423541     0.097989   

        C4_pct_Eth   C4_pct_H2O  HydWtr_pct_Ammonia   C4_pct_Hex  HydWtr_Na2O  \
count  4863.000000  4863.000000         4863.000000  4863.000000  4863.000000   
mean      1.802102    18.017469            0.948256     0.378261     1.689514   
min       0.565000    13.995000            0.

In [17]:
import pandas as pd

# Assuming the 'Date' column is a string, you can extract the year using the str accessor
# final_dataset['Year'] = final_dataset['Date'].str[:4]

# Alternatively, if the 'Date' column is already a datetime object, you can extract the year directly
final_dataset['Year'] = final_dataset['Date'].dt.year

# Now count the records for each year
record_counts_by_year = final_dataset['Year'].value_counts().sort_index()

# Output the results
print(record_counts_by_year)


Year
2023    4863
Name: count, dtype: int64


In [18]:
# Save DataFrame to CSV file in the same directory as the Jupyter Notebook
# model_results.to_csv('merged_data'.csv', index=False)
                     
# Save DataFrame to CSV file in the same directory as the Jupyter Notebook
# df_CD.to_csv(r'C:\Users\steve\OneDrive\1. BAIUTEK\Project-OptiC4\1 Preprocess\Continuous Data\contData_all.csv', index=False)            

final_dataset.to_csv(r'C:\Users\austinsh\Project-OptiC4\IV Optimize\Testing Data\Continuous Data 2023\merged_data.csv', index=False)


In [19]:
from datetime import datetime

# Get the current date and time
current_date_time = datetime.now()

# Print the current date and time
print(current_date_time)


2024-05-07 02:05:11.778789
